Environment

In [ ]:
import os
import sys
import importlib
if (importlib.util.find_spec("ronek") is None):
  sys.path.append("./../../../../ronek/")
sys.path.append("./../../")

from ronek import env
env.set(
  device="cpu",
  device_idx=0,
  nb_threads=16,
  floatx="float64"
)

Import libraries

In [ ]:
import numpy as np
import joblib as jl
import ronek.postproc.plotting as pltt

from utils import *
from tqdm import tqdm
from pyDOE import lhs
from ronek import const
from ronek.systems import TAFASystem

Define inputs
> System

In [ ]:
T = 1e4
inp_space = {
  "T": [5e2, 8e3],
  "p": [1e3, 1e4],
  "Xa": [0.05, 1.0]
}
tgrid = {
  "lim": [1e-12, 1e-2],
  "num": 501
}
nb_traj = 500
nb_workers = 32

> Paths

In [ ]:
paths = {
  "dtb": "./../database/",
  "data": "./data/"
}

Initialize isothermal master equation model

In [ ]:
model = TAFASystem(
  rates=paths["dtb"] + "/kinetics.hdf5",
  species={
    k: paths["dtb"] + f"/species/{k}.json" for k in ("atom", "molecule")
  },
  use_einsum=False
)
model.update_fom_ops(T)

Sample initial conditions

In [ ]:
def sample_inp_space(
  nb_traj,
  inp_space
):
  inp_space_lbl = list(inp_space.keys())
  inp_space = np.vstack(list(inp_space.values())).T
  # Contruct design matrix (DM)
  d = inp_space.shape[1]
  x = lhs(d, nb_traj)
  # Rescale DM
  x = x * (inp_space[1] - inp_space[0]) + inp_space[0]
  # Restructure DM
  x = [xi.squeeze() for xi in np.split(x, x.shape[-1], axis=-1)]
  x = {k: x[i] for (i, k) in enumerate(inp_space_lbl)}
  return np.vstack([x[k] for k in ("T", "p", "Xa")]).T

In [ ]:
ics = sample_inp_space(nb_traj, inp_space)

FOM solutions

In [ ]:
t = get_tgrid(tgrid["lim"], tgrid["num"])
sols = jl.Parallel(nb_workers)(
  jl.delayed(solve_fom)(model, t, get_y0(model, *ic)) for ic in tqdm(
    iterable=ics,
    ncols=80,
    desc="Box cases"
  )
)
y = np.hstack([sol[1]-sol[1][:,-1:] for sol in sols]) / const.UNA

POD
> SVD

In [ ]:
u, s, vh = sp.linalg.svd(y, full_matrices=False)

> Reconstruction

In [ ]:
def reconstruct(u, s, vh, r):
  return u[:,:r] @ np.diag(s[:r]) @ vh[:r]

def error(y_true, y_pred, eps=1e-8):
  return 100*np.mean(np.abs(y_true-y_pred)/(np.abs(y_true)+eps))

def plot_err(
  x,
  y,
  ylabel=r"$\Delta\tilde{c}$ [\%]",
  yscale="linear",
  figname=None,
  save=False,
  show=True
):
  # Initialize figure
  fig = plt.figure()
  ax = fig.add_subplot()
  # x axis
  ax.set_xlabel("$r$")
  ax.set_xlim([x[0]-1, x[-1]+1])
  # y axis
  ax.set_yscale(yscale)
  ax.set_ylabel(ylabel)
  # Plotting
  plt.hlines(1.0, x[0]-1, x[-1]+1, colors='grey', lw=1.0)
  ax.plot(x, y, marker="o")
  # Tight layout
  plt.tight_layout()
  if save:
    plt.savefig(figname)
  if show:
    plt.show()
  plt.close()

In [ ]:
err = []
dim = np.arange(5,26)
for r in dim:
  y_pred = reconstruct(u, s, vh, r)
  err.append(error(y, y_pred, eps=1e-8))

In [ ]:
path = paths["data"] + "/figs/"
os.makedirs(path, exist_ok=True)
plot_err(
  x=dim,
  y=err,
  ylabel=r"$\Delta\tilde{c}$ [\%]",
  yscale="log",
  figname=path + "pod",
  save=True,
  show=False
)